# Conformer finder using MM
With this method we generate all conformers by identifying rotatable bonds and rotate them according to their dihedral periodicities. 

In [ ]:
import veloxchem as vlx
import time
start_time = time.time()

We define our molecule and visualize it.

In [ ]:
#Lynparza
molecule = vlx.Molecule.read_name('paxlovid')
molecule.show()

This time we generate a forcefield using custom SCF settings.

In [ ]:
basis = vlx.MolecularBasis.read(molecule, 'def2-SVP')
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = "blyp"
scf_drv.ri_coulomb = True
scf_drv.grid_level = 2
scf_drv.conv_thresh = 1e-3
scf_drv.dispersion = True
scf_drv.timing = True
results = scf_drv.compute(molecule, basis)
resp_drv = vlx.RespChargesDriver()
charges = resp_drv.compute(molecule, basis, results, 'resp')

Now we can run the conformer generator. We just need to make sure to use our computed RESP charges from the previous cell.

In [ ]:
conf_generator = vlx.ConformerGenerator()
conf_generator.resp_charges = False
conf_generator.partial_charges = charges
conformers = conf_generator.generate(molecule)

We can visualize the lowest energy conformer.

In [ ]:
conf_generator.show_global_minimum()

FInaly we compute the energy of the new conformer to compare it to the initial structure.

In [ ]:
molecule = conformers['molecules'][0]
basis = vlx.MolecularBasis.read(molecule, 'def2-SVP')
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = "blyp"
scf_drv.ri_coulomb = True
scf_drv.grid_level = 2
scf_drv.conv_thresh = 1e-3
scf_drv.dispersion = True
results2 = scf_drv.compute(molecule, basis)

In [ ]:
print(f"New conformer is {(results2['scf_energy']-results['scf_energy'])*2625.5:.1f} kJ/mol lower in energy than the original structure")


In [ ]:
print(time.time()-start_time)